In [6]:
import random


In [7]:
import random
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import defaultdict

In [85]:

class LDA:
    def __init__(self, doc_ls, topic_num, alpha = 0.1, beta = 0.001):
        self.alpha = alpha
        self.beta = beta
        self.k = topic_num
        self.docs = doc_ls
    
    def RandomlyAssignTopic(self, doc_ls):
        dic = defaultdict(dict)
        t2i = defaultdict(lambda : len(t2i))
        i2t = defaultdict()
        d= 0
        w = 0
        wnl = WordNetLemmatizer()
        stopword = stopwords.words('english')
        stopword.append(',')



        for tokens in [word_tokenize(doc) for doc in doc_ls]:
            for token in [wnl.lemmatize(token.lower()) for token in tokens if token not in stopword]:
                i2t[t2i[token]] = token
                dic[(d, t2i[token], w)] = random.randint(0,self.k-1)
                w += 1
            d += 1
        #print(dic)
        return dic, t2i, i2t
    def CountDocTopic(self) :
        docs = np.zeros((self.k, len(self.docs)))
        terms = np.zeros((self.k, len(self.t2i.keys())))
        #문서별 토큰별 빈도수 계산
        for (d, n, w) in self.term_topic.keys() :
            docs[self.term_topic[(d, n, w)], d] += 1 + self.alpha
            terms[self.term_topic[(d, n, w)], d] += 1 + self.beta
        #print(doc_m)
        #비어있는 값는 값에 alpha, beta 설정
        docs = np.where(docs==0.0, self.alpha, docs)
        terms = np.where(terms==0.0, self.beta, terms)
        return docs, terms    
    def IterateAssignTopic(self, docs, terms) :
        #한개의 단어씩 주제 배정
        prev = {}
        while prev != self.term_topic:
            for (d, n, w) in self.term_topic.keys() :
                topic = [0, 0]
                docs[self.term_topic[(d, n, w)], d] -= (1 + self.alpha)
                terms[self.term_topic[(d, n, w)], n] -= (1 + self.beta)
                #print(doc_m)
                prev = self.term_topic
                for t in range(self.k) :
                    p_t_d = docs[t, d]/docs[:,d:d+1].sum()
                    p_w_t = terms[t, n]/terms[t:t+1,:].sum()
                    prob = p_t_d * p_w_t
                    if topic[1] < prob :
                        topic = [t, prob]
                #확률이 가장 큰 토픽을 할당
                self.term_topic[(d, n, w)] = topic[0]
                docs[self.term_topic[(d, n, w)], d] += (1 + self.alpha)
                terms[self.term_topic[(d, n, w)], t] += (1 + self.beta)
                #print(self.term_topic)
            return terms
# 토픽별 주요 키워드 출력
    def TopicModeling(self, count=5) :
        self.term_topic, self.t2i, self.i2t = self.RandomlyAssignTopic(self.docs)
        docs, terms = self.CountDocTopic()
        terms = self.IterateAssignTopic(docs, terms)
        score = terms / terms.sum(axis=1, keepdims=True)
        for i in range(self.k) :
            print("\nTopic {}".format(i+1))
            sorted_index = np.flip(np.argsort(score[i]),0)[:count]
            for j in sorted_index :
                print("({}={})".format(self.i2t[j], score[i,j].round(3)), end = ' ')    

    

In [92]:

doc_ls = ["Imagine waking up in your bedroom with no idea of where you are. Something feels familiar about the cotton sheets, the pictures on the wall, the sheer curtains, but you can't place it. Minutes later, you feel that same sensation of waking up, but this time you're standing at your dresser, wearing a T-shirt and jeans with no recollection of ever having been in bed. It's as though your consciousness lacks a past and a future, like a stop-motion film in which every previous frame is destroyed.",
'''But while people seem to easily remember tragic events and the seemingly insignificant details associated with them, many would be hard-pressed to recall the minutia of their happy times. For example, mothers often have trouble summoning the specifics of their children's birth, but are amazingly accurate in recounting the duration and intensity of the labor process. It begs the question, "Do we remember the bad times better than the good?" Before answering, it's helpful to know a bit about the process of memory formation and the factors that influence it.''',
"Kitty and hamster",
"Eat bread",
"Rice, bread and cake",
"Cute hamster eats bread and cake"]
lda = LDA(doc_ls, 4)
lda.TopicModeling()


Topic 1
(idea=1.039) (imagine=0.476) (waking=0.39) (people=0.0) (lack=0.0) 
Topic 2
(idea=1.028) (imagine=0.405) (waking=0.343) (people=0.0) ('s=0.0) 
Topic 3
(idea=0.998) (waking=0.304) (imagine=0.195) (something=0.065) (every=0.0) 
Topic 4
(waking=1.185) (idea=0.948) (imagine=0.711) (bedroom=0.047) (something=0.047) 

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)



11314

In [3]:
documents[:5]

["Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n",
 "\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap

In [4]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\15Z970-GA5BK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
news_df.head()a

,document,clean_doc
0,Well i'm not sure about the story nad it did s...,well sure about story seem biased what disagre...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re...",yeah expect people read actually accept hard a...
2,Although I realize that principle is not one o...,although realize that principle your strongest...
3,Notwithstanding all the legitimate fuss about ...,notwithstanding legitimate fuss about this pro...
4,"Well, I will have to change the scoring on my ...",well will have change scoring playoff pool unf...


In [8]:
tokenized_doc[0]

['well',
 'sure',
 'story',
 'seem',
 'biased',
 'disagree',
 'statement',
 'media',
 'ruin',
 'israels',
 'reputation',
 'rediculous',
 'media',
 'israeli',
 'media',
 'world',
 'lived',
 'europe',
 'realize',
 'incidences',
 'described',
 'letter',
 'occured',
 'media',
 'whole',
 'seem',
 'ignore',
 'subsidizing',
 'israels',
 'existance',
 'europeans',
 'least',
 'degree',
 'think',
 'might',
 'reason',
 'report',
 'clearly',
 'atrocities',
 'shame',
 'austria',
 'daily',
 'reports',
 'inhuman',
 'acts',
 'commited',
 'israeli',
 'soldiers',
 'blessing',
 'received',
 'government',
 'makes',
 'holocaust',
 'guilt',
 'away',
 'look',
 'jews',
 'treating',
 'races',
 'power',
 'unfortunate']

In [98]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0


[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [100]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.014*"compass" + 0.013*"scores" + 0.009*"toyota" + 0.007*"idle"')
(1, '0.020*"wiring" + 0.011*"helmet" + 0.009*"polygon" + 0.008*"cmos"')
(2, '0.016*"radar" + 0.011*"autos" + 0.010*"circuits" + 0.010*"detector"')
(3, '0.014*"available" + 0.010*"version" + 0.009*"also" + 0.009*"data"')
(4, '0.039*"file" + 0.022*"output" + 0.021*"program" + 0.020*"window"')
(5, '0.014*"guns" + 0.010*"control" + 0.009*"crime" + 0.009*"states"')
(6, '0.010*"jesus" + 0.007*"believe" + 0.007*"would" + 0.006*"people"')
(7, '0.039*"color" + 0.022*"colors" + 0.013*"colormap" + 0.012*"mask"')
(8, '0.011*"year" + 0.011*"game" + 0.010*"team" + 0.008*"good"')
(9, '0.032*"jpeg" + 0.019*"format" + 0.018*"image" + 0.012*"shareware"')
(10, '0.019*"printf" + 0.010*"soon" + 0.009*"gordon" + 0.009*"pitt"')
(11, '0.011*"information" + 0.009*"space" + 0.008*"mail" + 0.007*"send"')
(12, '0.019*"period" + 0.015*"play" + 0.011*"power" + 0.010*"pittsburgh"')
(13, '0.028*"health" + 0.021*"medical" + 0.016*"disease" + 0.013

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)